In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
import os, re, struct
import pandas as pd
import numpy as np
# manually list out the IDs for series of interest 
#var_names = ['HRHHID','HRHHID2','PULINENO','PRPERTYP','PRTAGE','PEMLR','PESEX', 'PTDTRACE','PRDTOCC1', 'PWCMPWGT','HWHHWGT']
#var_names = ['PEHRUSL1', 'PEHRUSL2', 'PEHRACT1', 'PEHRACT2', 'PEERNHRO', 'PTERNWA', 'PWORWGT', 'PEEDUCA', 'PRNMCHLD', 'PEMARITL', 'PRDTIND1', 'PRMJIND1','PRERNWA']
var_names = ['PEHRUSL1', 'PEHRUSL2', 'PEHRACT1', 'PEHRACT2', 'PEERNHRO', 'PTERNWA', 'PWORWGT', 'PEEDUCA', 'PRNMCHLD', 'PEMARITL', 'PRDTIND1', 'PRMJIND1','PRERNWA',\
             'PRPERTYP','PRTAGE','PEMLR','PESEX', 'PTDTRACE','PRDTOCC1', 'PWCMPWGT','PEAGE','PRDTRACE','PERACE','PEHRFTPT', 'HRINTSTA', 'PRDTCOW1', 'PEIO1COW', 'HRMIS', 'PEMJOT']
required_names=['PRPERTYP','PRTAGE','PEMLR','PESEX', 'PTDTRACE','PRDTOCC1', 'PWCMPWGT','PEHRUSL1', 'PEHRUSL2', 'PEHRACT1', 'PEHRACT2', 'PEERNHRO', 'PTERNWA', 'PWORWGT',\
                'PEEDUCA', 'PRNMCHLD', 'PEMARITL', 'PRDTIND1', 'PRMJIND1','PEHRFTPT', 'HRINTSTA', 'PRDTCOW1', 'PEIO1COW', 'HRMIS', 'PEMJOT']
required_names.sort()
calendarMap={'jan':'01','feb':'02','mar':'03','apr':'04','may':'05','jun':'06','jul':'07','aug':'08','sep':'09','oct':'10','nov':'11','dec':'12'}



Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
len(required_names)

25

In [ ]:
import copy
from_path='/content/drive/My Drive/BOC_V2/dictionaries for all years/'
files=os.listdir(from_path)
dataList=[]
dataList_Nonexisted=[]
specialFiles=[]
for fileName in files:
  file=from_path+fileName
  # read data dictionary text file 
  data_dict = open(file).read()
  #2021dictionary0112
  p0='([0-9]+)dictionary([0-9]+)'
  d0=re.findall(p0, fileName)[0]
  year, month_start, month_end = int(d0[0]), int(d0[1][0:2]), int(d0[1][2:4])
  if year>=2003:
    # regular expression matching series name and data dict pattern
    #(HRHHID|HRMONTH)\s+[0-9]*\s+.*\S\s\s+[(]*([0-9]+)\s*.\s*([0-9]+)[)]*?
    #(PRDTIND1|PRDTCOW1|PRNMCHLD)\s+\d*\s+.*\S\s[\s+|(]*(\d+)\s*[-]\s*(\d+)[)]*?
    var_names1=copy.deepcopy(var_names)
    var_names1.remove('PRDTCOW1')
    p = f'({"|".join(var_names1)})\s+[0-9]*\s+.*\S\s\s+[(]*([0-9]+)\s*.\s*([0-9]+)[)]*?'
    d = [[year, month_start, month_end, s[0], int(s[1])-1, int(s[2])] for s in re.findall(p, data_dict)] #year, month_start, month_end, name, start, end
    names = {s[0]: [int(s[1])-1, int(s[2])]
      for s in re.findall(p, data_dict)}
    
    if d and names:
      dataList.extend(d)
      if len(d) != len(var_names1)-2:
        
        availableNames=list(names.keys())

        not_availableNames=[name for name in var_names1 if name not in availableNames]

        dataList_Nonexisted.append([fileName, availableNames, not_availableNames])

    else:
      specialFiles.append(fileName)

    p_PRDTCOW1 = 'PRDTCOW1\s+\d*\s+.*\S\s[\s+|(]*(\d+)\s*[-]\s*(\d+)[)]*?'
    try:
      s=re.findall(p_PRDTCOW1, data_dict)[0]
    except:
      print(fileName)
    #print(f's is {s}')  s is ('476', '477')
    if s:
      dataList.append([year, month_start, month_end, 'PRDTCOW1', int(s[0])-1, int(s[1])])
    else:
      print(f"didn't find the location of PRDTCOW1")



In [ ]:
df=pd.DataFrame(dataList)
df.columns=['year','month_start', 'month_end','name','location_start', 'location_end']
print(len(df))
df.head()

550


,year,month_start,month_end,name,location_start,location_end
0,2021,1,12,HRINTSTA,56,58
1,2021,1,12,HRMIS,62,64
2,2021,1,12,PRTAGE,121,123
3,2021,1,12,PEMARITL,124,126
4,2021,1,12,PESEX,128,130


In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth',100)

df.replace('PRERNWA','PTERNWA',inplace=True)
df.replace('PEAGE','PRTAGE', inplace=True)
df.replace('PRDTRACE', 'PTDTRACE', inplace=True)
df.replace('PERACE', 'PTDTRACE', inplace=True)
df.sort_values(['year','month_start','name'],inplace=True)
df_sub=df.loc[(df['year']>=2003)].drop_duplicates(subset=['location_start'], keep='first', inplace=False, ignore_index=False)
df_sub.sort_values(['location_start'])

,year,month_start,month_end,name,location_start,location_end
475,2003,1,12,HRINTSTA,56,58
476,2003,1,12,HRMIS,62,64
477,2003,1,12,PRTAGE,121,123
478,2003,1,12,PEMARITL,124,126
479,2003,1,12,PESEX,128,130
480,2003,1,12,PEEDUCA,136,138
481,2003,1,12,PTDTRACE,138,140
482,2003,1,12,PRPERTYP,160,162
483,2003,1,12,PEMLR,179,181
484,2003,1,12,PEMJOT,213,215


In [ ]:
df_sub

,year,month_start,month_end,name,location_start,location_end
475,2003,1,12,HRINTSTA,56,58
476,2003,1,12,HRMIS,62,64
480,2003,1,12,PEEDUCA,136,138
494,2003,1,12,PEERNHRO,524,526
488,2003,1,12,PEHRACT1,242,244
489,2003,1,12,PEHRACT2,244,246
487,2003,1,12,PEHRFTPT,221,223
485,2003,1,12,PEHRUSL1,217,219
486,2003,1,12,PEHRUSL2,219,221
490,2003,1,12,PEIO1COW,431,433


In [ ]:
missed_name=[i for i in required_names if i not in df_sub['name'].to_list()]
missed_name

[]

In [ ]:
df_analysis_2003_all=df[df['year']>=2003]
df_analysis_2003_all.sort_values(['year','month_start','name'],inplace=True)
df_analysis_2003_all.reset_index(drop=True,inplace=True)
df_analysis_2003_all.head()

,year,month_start,month_end,name,location_start,location_end
0,2003,1,12,HRINTSTA,56,58
1,2003,1,12,HRMIS,62,64
2,2003,1,12,PEEDUCA,136,138
3,2003,1,12,PEERNHRO,524,526
4,2003,1,12,PEHRACT1,242,244


In [ ]:
count=0
for year in df_analysis_2003_all['year'].unique():
  name_list = df_analysis_2003_all[df_analysis_2003_all['year']==year]['name'].to_list()
  name_list.sort()
  missed_name=[i for i in required_names if i not in name_list]
  if missed_name:
    print(year)
    print(missed_name)
    count+=1
if count==0:
  print('all required names existed for each year')

all required names existed for each year


Test the sub dataframe: ['PRPERTYP','PRTAGE','PEMLR','PESEX', 'PTDTRACE','PRDTOCC1', 'PWCMPWGT'] with previous output

In [ ]:
import pickle
df_analysis_2003_sub=pickle.load(open('/content/drive/My Drive/BOC_V2/df_analysis_2003','rb'))

In [ ]:
df_analysis_2003_sub.sort_values(['year','month_start','name'],inplace=True)
df_analysis_2003_sub.drop(['variable_length'], inplace=True, axis=1)
df_analysis_2003_sub.reset_index(drop=True, inplace=True)
df_analysis_2003_sub.head()

,year,month_start,month_end,name,location_start,location_end
0,2003,1,12,PEMLR,179,181
1,2003,1,12,PESEX,128,130
2,2003,1,12,PRDTOCC1,475,477
3,2003,1,12,PRPERTYP,160,162
4,2003,1,12,PRTAGE,121,123


In [ ]:
test_names=['PRPERTYP','PRTAGE','PEMLR','PESEX', 'PTDTRACE','PRDTOCC1', 'PWCMPWGT']
df_analysis_2003_sub_test=df_analysis_2003_all[df_analysis_2003_all['name'].isin(test_names)]
df_analysis_2003_sub_test.sort_values(['year','month_start','name'],inplace=True)
df_analysis_2003_sub_test.reset_index(drop=True, inplace=True)
df_analysis_2003_sub_test.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,year,month_start,month_end,name,location_start,location_end
0,2003,1,12,PEMLR,179,181
1,2003,1,12,PESEX,128,130
2,2003,1,12,PRDTOCC1,475,477
3,2003,1,12,PRPERTYP,160,162
4,2003,1,12,PRTAGE,121,123


In [ ]:
df_analysis_2003_sub.equals(df_analysis_2003_sub_test)

True

In [ ]:
import pickle
pickle.dump(df_analysis_2003_all, open('/content/drive/My Drive/BOC_V2/df_analysis_2003_all_V4','wb'))

In [ ]:
df_analysis_2003_all

,year,month_start,month_end,name,location_start,location_end
0,2003,1,12,HRINTSTA,56,58
1,2003,1,12,HRMIS,62,64
2,2003,1,12,PEEDUCA,136,138
3,2003,1,12,PEERNHRO,524,526
4,2003,1,12,PEHRACT1,242,244
5,2003,1,12,PEHRACT2,244,246
6,2003,1,12,PEHRFTPT,221,223
7,2003,1,12,PEHRUSL1,217,219
8,2003,1,12,PEHRUSL2,219,221
9,2003,1,12,PEIO1COW,431,433


In [ ]:
abnormal_location_name=[]
for index, row in df_analysis_2003_all.iterrows():
  if row['location_start']>=row['location_end']:
    abnormal_location_name.append(row['name'])
if abnormal_location_name:
  print(f'abnormal location names: {abnormal_location_name}')
else:
  print('all location_end > location_start')

all location_end > location_start
